# Exercício de avaliação 1 - Relatório/Notebook

## Enunciado do primeiro exercício de avaliação:

**1.** Use a package cryptography    para  criar um comunicação privada assíncrona entre um agente Emitter e um agente Receiver que cubra os seguintes aspectos: \
    **a.** Comunicação cliente-servidor que use o package python `asyncio`.\
    **b.** Usar como cifra AEAD   o “hash” SHAKE-256  em modo XOFHash \
    **c.** As chaves de cifra  e  os “nounces” são gerados por um gerador KDF . As diferentes chaves para inicialização KDF  são inputs do emissor e do receptor.

## Introdução

A segurança na comunicação digital é essencial para garantir a confidencialidade e a integridade dos dados trocados entre agentes. Neste trabalho, implementamos um canal de comunicação privada entre um `Emitter` e um `Receiver`, garantindo a proteção das mensagens através de criptografia autenticada (AEAD) e mecanismos seguros de derivação de chaves.

A comunicação é baseada no modelo cliente-servidor assíncrono, utilizando o pacote `asyncio` do *Python* para permitir a troca de mensagens entre os agentes. O esquema de criptografia utilizado foi **_SHAKE-256_** em modo **_XOFHash_** como **_Authenticated Encryption with Associated Data_** (AEAD), garantindo tanto a confidencialidade quanto a integridade das mensagens trocadas.

As chaves de cifra e os nonces necessários para a encriptação e desencriptação das mensagens são gerados através de um **_Key Derivation Function_** (KDF). As chaves de inicialização do KDF são fornecidas como inputs pelo emissor e pelo receptor, garantindo que ambos os agentes possam derivar as mesmas chaves de forma independente e segura.

Para estruturar a implementação, tornar mais claro o que é nos pedido para chegar a uma implementação final e garantir que o sistema cumpra os objetivos definidos, foram determinados requisitos funcionais, estes definem as funcionalidades essenciais que o sistema deve oferecer:

## Requisitos funcionais

**RF1** - Deve haver uma lógica de cifraçãode mensagens através da cifra AEAD com hash SHAKE-256 em modo XOFHash\
**RF2** - Deve haver uma lógica de decifração de mensagens através da cifra AEAD com hash SHAKE-256 em modo XOFHash\
**RF3** - O servidor deve verificar a tag de autenticação antes de processar a mensagem do cliente \
**RF4** - Para efeitos de modularidade e para que a lógica de cifração não esteja contida nos ficheiros de cliente e servidor deve-se criar um ficheiro à parte com as funções de cifração \
**RF5** - O servidor fica à escuta de conexões do cliente \
**RF6** - O servidor deve ser capaz de receber mensagens criptografadas \
**RF7** - O servidor deve guardar as conexões com cada utilizador guardando e alterando informação relevante para que as novas mensagens enviadas nunca sejam iguais no processo de comunicação \
**RF8** - O cliente quando inicializa fica à escuta para receber parâmetros relevantes para a definição da chave e do nonce \
**RF9** - O cliente envia mensagens cifradas dado o protocolo de cifragem \
**RF10** - O cliente quando envia a sua mensagem cifrada deve mandar também a tag de autenticação \
**RF11** - Após envio de mensagem o cliente deve atualizar os parâmetros relevantes (de maneira igual ao servidor) para obter imunidade contra ataques de replicação \
**RF12** - Os nonce's e chaves de cifra dos clientes são definidos por KDF 

Neste documento iremos mostrar como é que foram satisfeitos todos os requisitos funcionais estabelecidos explicando pretinentemente o raciocínio por de trás da implementação.

----------------------------------------------------------

## **RF1** -  Deve haver uma lógica de cifração de mensagens através da cifra AEAD com hash SHAKE-256 em modo XOFHash

Para satisfazer este requisito funcional o grupo decidiu seguir o plano de implementação definido no capítulo 1 "Primitivas Criptográficas Básicas" mais concretamente uma **Cifra AEAD usando o modelo "sponge"**.

![Cifra AEAD usando o modelo "sponge"](Sponge.png)

Começamos por definir o estado inicial da função concatenando o vetor inicial (nonce) com a chave de cifração (dada por *input* do cliente e do servidor), assim como a função de Hash, neste caso **_SHAKE256_**, uma função de hash baseada na família SHA-3 de output extensível (XOFHash), neste caso de output de 256 bits fixo. Esta função de hash é derivada de uma das famílias de hash mais seguras atualmente como podemos observar nesta tabela de tempos de vida de hashes criptográficos populares:

![Lifetimes of popular cryptographic hashes (the rainbow chart)](HashesLifetime.png)

(fonte : https://valerieaurora.org/hash.html)

Após isso entramos numa 1ª fase **“absorb”** são absorvidos os segmentos que descrevem os dados associados.

Na 2ª fase, equivalente ao **“squeeze”**, usa-se uma cifra que recebe segmentos $\,p_i\,$ sucessivos de um “_plaintext_” , soma-os com $r$ bits do estado e devolve os segmento $\,c_i\,$ do criptograma.

Finalmente, após o processamento de todo o “plaintext” existe uma na aplicação da permutação para construir a “tag” de autenticação final.
A chave $K$ da cifra, aparece na inicialização do 1º valor do estado e aparece antes e depois da última aplicação da permutação.


### O processo todo é na prática implementado da seguinte forma:

**Passo 1:** Definição de estado inicial aplicando a função de hash

In [1]:
import os
from cryptography.hazmat.primitives import hashes

#Input aleatório de ambos os parâmetros (o input real do programa é dado de maneira diferente)
key = os.urandom(32)
nonce = os.urandom(16)


iv_key = nonce + key   
shake = hashes.Hash(hashes.SHAKE256(64))

shake.update(iv_key)

**Passo 2:** Fase de **absorb** dos dados associados 

In [2]:
#Exemplo de dados associados
associated_data = b'Isto sao dados associados que precisam ser processados em blocos.'
block_size = 16
for i in range(0, len(associated_data), block_size):
    shake.update(associated_data[i:i + block_size])

**Passo 3:** Processar a mensagem e fazer **squeeze** do _ciphertext_

In [3]:
#Exemplo de plaintext
plaintext = b"Ola servidor! Espero que esta mensagem esteja cifrada!"

ciphertext = b""
for i in range(0, len(plaintext), block_size):
    block = plaintext[i:i + block_size]
    keystream = shake.copy().finalize()[:len(block)]
    ciphertext += bytes(a ^ b for a, b in zip(block, keystream))
    print(f"[CLIENTE] Absorvendo bloco: {block.hex()}")
    print(f"(Bloco original: ",block, ")")
    shake.update(block)

[CLIENTE] Absorvendo bloco: 4f6c61207365727669646f7221204573
(Bloco original:  b'Ola servidor! Es' )
[CLIENTE] Absorvendo bloco: 7065726f207175652065737461206d65
(Bloco original:  b'pero que esta me' )
[CLIENTE] Absorvendo bloco: 6e736167656d20657374656a61206369
(Bloco original:  b'nsagem esteja ci' )
[CLIENTE] Absorvendo bloco: 667261646121
(Bloco original:  b'frada!' )


**Passo 4:** Finalizar e gerar a _tag_ de autenticação

In [4]:
    #Passo 4: Finaliza e gera a Tag
    state = shake.copy().finalize()[:block_size]

    # XOR com K
    state = bytes(a ^ b for a, b in zip(state, key))
    # Aplica hash com esse novo estado
    shake.update(state)
    # Aplica um segundo XOR com K
    state = bytes(a ^ b for a, b in zip(state, key))

    # Finaliza tag
    tag = shake.finalize()[:block_size]

    print ("ciphertext: ",ciphertext)
    print ("tag: ",tag)

ciphertext:  b'\x92\xae\xc3%\xc3\x15L\x02=\xbb.\x85\xe9E^0\xf6W\xdf\xf6TX\xa9y\xd0F\x91\xb4\xc2\x93\x98\xc8\xeb"\xb0\xce\xda\xfdr9\xc9\x93\xe6\x19\x0e\x12]\x85QZ)sF\xc0'
tag:  b'\x18\x80\xeb\xa7\x84,\x9e\x9b\xed\xf6\xe6\x8a\x03x\x86!'


-------------------------------------------------------------------------------------------------------------------------------------------------

## **RF2** - Deve haver uma lógica de decifração de mensagens através da cifra AEAD com hash SHAKE-256 em modo XOFHash

Para decifrar a mensagem o processo é semelhante, queremos criar um fluxo de chave igual (possível dado que temos a chave de cifração e o nonce) e a partir dela aplicar analogamente o processo de absorção ao texto cifrado, obtendo o texto correto (_plaintext_), após isso também finalizamos gerando a tag de autenticação, se esta corresponder à tag recebida (abordado no **RF3**) é relativamente seguro prosseguir:

In [5]:
    iv_key = nonce + key
    shake = hashes.Hash(hashes.SHAKE256(64))
    
    # Passo 1: Iniciar com IV || Key e aplicar o shake
    shake.update(iv_key)

    # Passo 2: Absorver os dados associados 
    for i in range(0, len(associated_data), block_size):
        shake.update(associated_data[i:i + block_size])

    # Passo 3: Processar o ciphertext e recuperar o plaintext
    plaintext = b""
    for i in range(0, len(ciphertext), block_size):
        block = ciphertext[i:i + block_size]
        keystream = shake.copy().finalize()[:len(block)]
        plaintext += bytes(a ^ b for a, b in zip(block, keystream))
        #print(f"[SERVIDOR] Absorvendo bloco: {block.hex()}")
        #shake.update(block)  # Continua absorvendo
        shake.update(plaintext[i:i + block_size])  # Absorver o plaintext correto

    # Passo 4: Recalcular a Tag para verificar a integridade
    state = shake.copy().finalize()[:block_size]  # Obtém estado interno atual

    #print(f"[SERVIDOR] Estado após primeiro XOR: {state.hex()}")

    # XOR com K
    state = bytes(a ^ b for a, b in zip(state, key))
    # Faz update com esse novo estado
    shake.update(state)
    # Aplica um segundo XOR com K
    state = bytes(a ^ b for a, b in zip(state, key))
    
    # Finaliza e gera a tag para verificação
    #print(f"[SERVIDOR] Estado final antes da tag: {shake.copy().finalize().hex()}")

    computed_tag = shake.finalize()[:block_size]

-------------------------------------------------------------------------------------------------------------------------------------------------

## **RF3** - O servidor deve verificar a tag de autenticação antes de processar a mensagem do cliente

Após decifração (e devida cifração) a tag final gerada deve corresponder à tag recebida pelo servidor do cliente

In [6]:
if computed_tag != tag:
    raise ValueError("Autenticação falhou! O ciphertext foi alterado ou a chave está errada.")

-------------------------------------------------------------------------------------------------------------------------------------------------

## **RF4** -  Para efeitos de modularidade e para que a lógica de cifração não esteja contida nos ficheiros de cliente e servidor deve-se criar um ficheiro à parte com as funções de cifração

Como é possível comprovar a pasta contém um ficheiro **crypto_utils** onde está contida toda a lógica de cifração.

O grupo 7 decidiu manter a cifração num módulo à parte evitando que lógicas sensíveis fiquem expostas ou misturadas com código de rede, reduzindo a possibilidade de vulnerabilidades. Para além disso o código de servidor e cliente fica mais legível e há uma maior reutilização de código se necessário no futuro.

-------------------------------------------------------------------------------------------------------------------------------------------------

## **RF5** - O servidor fica à escuta de conexões do cliente

Para satisfazer este requisito utilizamos a biblioteca `asyncio` para definir uma função que irá ficar à escuta no IP local da máquina na porta 8888 até que o processo seja morto.  

In [7]:
async def start_server(self):
    server = await asyncio.start_server(self.handle_client, '127.0.0.1', 8888)
    async with server:
        await server.serve_forever() 

-------------------------------------------------------------------------------------------------------------------------------------------------

## **RF6** - O servidor deve ser capaz de receber mensagens criptografadas

A lógica de receção de mensagens é se o servidor não estiver no "`End of file`", ou seja, até nós não matarmos o processo. Dado o nosso protocolo queremos ler os primeiros 4 bytes para definição do `ID` de um novo cliente. 

Para não estarmos sujeitos a ficar bloquados no `await` criámos certas exceções para tratar de tal ocasião:

In [8]:
async def receive_messages_example(reader):
    try:
        while not reader.at_eof():
            id_bytes = await asyncio.wait_for(reader.readexactly(4), timeout=5)
            print("######## MENSAGEM RECEBIDA ########")
    except asyncio.TimeoutError:
        print("Nenhuma mensagem recebida dentro do tempo limite.")
    except asyncio.IncompleteReadError:
        print("Conexão fechada antes de receber os 4 bytes esperados.")

-------------------------------------------------------------------------------------------------------------------------------------------------

## **RF7** - O servidor deve guardar as conexões com cada utilizador guardando e alterando informação relevante para que as novas mensagens enviadas nunca sejam iguais no processo de comunicação 

Foi utilizada uma função de chave derivada (KDF) fazendo uso da primitiva `HKDF` que como algoritmo de hash utiliza o `SHAKE-256`:

In [9]:
from cryptography.hazmat.primitives.kdf.hkdf import HKDF

def hkdf_sha256(salt: bytes, key: bytes) -> bytes:
    hkdf = HKDF(
        algorithm=hashes.SHA256(),
        length=44, # 32 bytes para a chave e 12 bytes para o nonce
        salt=salt, #Randomizes the KDF’s output.
        info=b'',
    )
    derived_bytes = hkdf.derive(key)
    return derived_bytes

#Exemplo de um cliente
client_id = b"3"
message_id = b"5"
salt = b"salt_clientid_" + str(client_id).encode() + b"_messageid_" + str(message_id).encode()

derived_bytes = hkdf_sha256(salt, key)
print(derived_bytes)

b'3\xf9h\xc0s\x8b\x9b\x85\x04_\x0f\\\xb2v\xe0io\xb6\xf6\xc4\xa2q\x9bL/\xec4\x84\xf6<2\x9d\xe5g\x1e\x0fN\xf6\xa2\xf9f\xce\xc8\x9b'


In [10]:
clients = {}
clients[int(client_id)] = {
    "writer": "writer",
    "cypher_key": derived_bytes[:32], # 32 bytes para a chave -> 256 bits
    "nonce": derived_bytes[32:], # 12 bytes para o nonce -> 96 bits
    "messageCounter": 0
}

Após envio de uma mensagem de um cliente a lógica implementada para que as próximas mensagens não tenham um criptograma igual é a seguinte (tanto para cliente e servidor): 

In [11]:
clients[int(client_id)]['messageCounter'] += 1

In [12]:
def enviadados():
    salt = b"salt_clientid_" + str(client_id).encode() + b"_messageid_" + str(self.clients[int(client_id)]['messageCounter']).encode()
    writer.write(salt + b"\n")  # Envia o salt para o cliente
    #await writer.drain()
    derived_bytes = hkdf_sha256(salt, self.server_key)
    self.clients[int(client_id)] = {
                    "writer": writer,
                    "cypher_key": derived_bytes[:32], # 32 bytes para a chave -> 256 bits
                    "nonce": derived_bytes[32:], # 12 bytes para o nonce -> 96 bits
                    "messageCounter": self.clients[int(client_id)]['messageCounter']
                }

-------------------------------------------------------------------------------------------------------------------------------------------------

## **RF8** - O cliente quando inicializa fica à escuta para receber parâmetros relevantes para a definição da chave e do nonce 

Definido no `Client.py` temos toda a lógica de conexão com o servidor:

        reader, writer = await asyncio.open_connection('127.0.0.1', 8888)

        # Recebe o client_id
        client_id = await reader.readline()
        client_id = client_id.decode().strip()
        print(f"Cliente ID recebido: {client_id}")

        # Recebe o salt
        salt = await reader.readline()
        salt = salt.strip()  # Remove o '\n'
        print(f"Salt recebido: {salt.decode()}")

        # Deriva a chave e nonce com HKDF
        derived_bytes = hkdf_sha256(salt, self.client_key)

        self.cypher_key = derived_bytes[:32] # 32 bytes para a chave -> 256 bits
        self.client_nonce = derived_bytes[32:] # 12 bytes para o nonce -> 96 bits
        print(f"Chave da cifra: {self.cypher_key.hex()}")
        print(f"Nonce do cliente: {self.client_nonce.hex()}")

Nomeadamente temos sequencialmente: \
     - Conexão com o servidor; \
     - Receção de informação do seu número de identificação; \
     - Receção de dados relevantes para definição de `chave de cifra` e de `nonce`. \
Após estes passsos o cliente fica apto para enviar mensagens de forma segura para o servidor.

-------------------------------------------------------------------------------------------------------------------------------------------------

## **RF9** - O cliente envia mensagens cifradas dado o protocolo de cifragem

O código escrito no `Client.py` faz uso das primitivas definidas definidas no ficheiro `crypo_utils.py`, nomeadamente `hkdf_sha256` para definição de chave de cifra e nonce assim como o `sponge_aead_encrypt` para cifração de mensagem dados esses mesmo parâmetros juntamente com a mensagem a ser cifrada.

-------------------------------------------------------------------------------------------------------------------------------------------------

## **RF10** - O cliente quando envia a sua mensagem cifrada deve mandar também a tag de autenticação

Dado no requisito funcional anterior, nomeadamente em:

Ou seja, o cliente lança para o servidor uma mensagem de corpo:
 - ID do cliente || mensagem cifrada || tag

(Onde || é a função usual de concatenação)

-------------------------------------------------------------------------------------------------------------------------------------------------

## **RF11** - Após envio de mensagem o cliente deve atualizar os parâmetros relevantes (de maneira igual ao servidor) para obter imunidade contra ataques de replicação

O parâmetro `salt` é recebido do servidor tanto como garantia que a mensagem foi recebida como para definir novos valores para a chave de `cifra` e o `nonce`, tentando ao máximo não ser previsível.

-------------------------------------------------------------------------------------------------------------------------------------------------

## **RF12** - Os nonce's e chaves de cifra dos clientes são definidos por KDF 

Através da primitva definida `hkdf_sha256` no ficheiro de `crypto_utils.py`:

In [13]:
def hkdf_sha256(salt: bytes, key: bytes) -> bytes:
    hkdf = HKDF(
        algorithm=hashes.SHA256(),
        length=44, # 32 bytes para a chave e 12 bytes para o nonce
        salt=salt, #Randomizes the KDF’s output.
        info=b'',
    )
    derived_bytes = hkdf.derive(key.encode())
    return derived_bytes

Conseguimos definir por KDF, a `chave de cifra` (primeiros 32 bytes) e o `nonce` (últimos 12 bytes)

Que irão ser armazenados no servidor como informação relevante da conexão com o cliente

-------------------------------------------------------------------------------------------------------------------------------------------------

## Conclusão e Observações

O grupo considera que comunicação privada assíncrona foi implementada corretamente utilizando as biblioteca de `asyncio` para estabelecer a comunicação **cliente-servidor** entre os agentes `Emitter` e `Receiver`. Para a cifra **AEAD**, foi empregado o **SHAKE-256** no modo **XOFHash**, aproveitando as propriedades do **modelo sponge** para gerar uma saída variável e resistente a ataques de pré-imagem.

As **chaves de cifra** e os **nonces** foram derivados de um **KDF** (Key Derivation Function), garantindo unicidade e imprevisibilidade para cada sessão, características essenciais a cumprir nas estruturas criptográficas. 

O grupo considera que a estratégia de acordo de informação e parâmetros ed cifração e decifração foram bem implementados e que deixa pouca margem a ataques.

------

### **Ficheiro _Notebook_ do trabalho prático 1 - Exercício de avaliação alínea 1**
### Estruturas Criptográficas 2024/2025 

#### Realizado pelo grupo 7:
Paulo André Alegre Pinto PG55991 \
Pedro Miguel Dias Leiras PG55995